In [1]:
#DEPENDENCIES
import pandas as pd
import pandas_read_xml as pdx
from bs4 import BeautifulSoup
import requests
import pymongo
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager


In [2]:
url = "https://mars.nasa.gov/news/?page=0&per_page=10&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"

In [4]:
# Retrieve page with the requests module
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - There is no [win32] chromedriver for browser 89.0.4389 in cache
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/89.0.4389.23/chromedriver_win32.zip


[WDM] - Driver has been saved in cache [C:\Users\pwitw\.wdm\drivers\chromedriver\win32\89.0.4389.23]


In [4]:
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [18]:
news_date = soup.find('div', class_='list_date').text == 'December  8, 2020'
news_date

True

In [19]:
news = soup.find_all('div', attrs={'class':'list_date'})
news_title = ''
for div in news:
    try:
        if div.find('div', attrs={'list_date'}) == 'December  8, 2020' :
            nextDiv = div.findNext('div', attrs={'class': 'content_title'})
            news_title= nextDiv.find('a').text
    except AttributeError:
        print('attribute missing')

print(news_title)
#getting the data div using findNext
#nextDiv =  div.findNext('div', attrs={'class':'data'})
#use findNext again on previous object to get to the anchor tag
#fileName = nextDiv.findNext('a').text

In [21]:
mars_news = soup.find('ul', class_='item_list')
news_title = mars_news.find('div', class_='content_title').text
news_title

"5 Hidden Gems Are Riding Aboard NASA's Perseverance Rover"

In [23]:
news_p = mars_news.find('div', class_='article_teaser_body').text
news_p

"The symbols, mottos, and small objects added to the agency's newest Mars rover serve a variety of purposes, from functional to decorative."

In [25]:
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [26]:
full_image = browser.find_by_id('full_image')
full_image.click()

In [27]:
browser.is_element_present_by_text('more info')
more_info = browser.links.find_by_partial_text('more info')
more_info.click()

In [32]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
main_image = soup.find('img', class_='main_image')['src']
main_image
#https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA19324_hires.jpg
url = f"https://www.jpl.nasa.gov{main_image}"
print(url)

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA19324_hires.jpg


In [40]:
url = 'https://space-facts.com/mars/'
mars_facts_df = pd.read_html(url)[0]
mars_facts_df.columns=['Description', 'Facts']
mars_facts_df.set_index('Description', inplace = True)
mars_facts_df.head()


,Facts
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"


In [49]:
#construct hemisphere dictionary

url= 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')



In [51]:
hemis = soup.find_all('a', class_='itemLink product-item')
hemis

[<a class="itemLink product-item" href="/search/map/Mars/Viking/cerberus_enhanced"><img alt="Cerberus Hemisphere Enhanced thumbnail" class="thumb" src="/cache/images/39d3266553462198bd2fbc4d18fbed17_cerberus_enhanced.tif_thumb.png"/></a>,
 <a class="itemLink product-item" href="/search/map/Mars/Viking/cerberus_enhanced"><h3>Cerberus Hemisphere Enhanced</h3></a>,
 <a class="itemLink product-item" href="/search/map/Mars/Viking/schiaparelli_enhanced"><img alt="Schiaparelli Hemisphere Enhanced thumbnail" class="thumb" src="/cache/images/08eac6e22c07fb1fe72223a79252de20_schiaparelli_enhanced.tif_thumb.png"/></a>,
 <a class="itemLink product-item" href="/search/map/Mars/Viking/schiaparelli_enhanced"><h3>Schiaparelli Hemisphere Enhanced</h3></a>,
 <a class="itemLink product-item" href="/search/map/Mars/Viking/syrtis_major_enhanced"><img alt="Syrtis Major Hemisphere Enhanced thumbnail" class="thumb" src="/cache/images/55a0a1e2796313fdeafb17c35925e8ac_syrtis_major_enhanced.tif_thumb.png"/></a>,

In [72]:
href_list = []
for i in soup.find_all('a', class_= 'itemLink product-item'):
    print(i['href'])
    href_list.append(i['href'])


/search/map/Mars/Viking/cerberus_enhanced
/search/map/Mars/Viking/cerberus_enhanced
/search/map/Mars/Viking/schiaparelli_enhanced
/search/map/Mars/Viking/schiaparelli_enhanced
/search/map/Mars/Viking/syrtis_major_enhanced
/search/map/Mars/Viking/syrtis_major_enhanced
/search/map/Mars/Viking/valles_marineris_enhanced
/search/map/Mars/Viking/valles_marineris_enhanced


In [73]:
test_set = []
[test_set.append(x) for x in href_list if x not in test_set]
test_set

['/search/map/Mars/Viking/cerberus_enhanced',
 '/search/map/Mars/Viking/schiaparelli_enhanced',
 '/search/map/Mars/Viking/syrtis_major_enhanced',
 '/search/map/Mars/Viking/valles_marineris_enhanced']

In [76]:
base_url = 'https://astrogeology.usgs.gov/'

browser.visit(base_url + test_set[0])
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [81]:
names = soup.find('title').text
names

'Cerberus Hemisphere Enhanced | USGS Astrogeology Science Center'

In [85]:
img_urls = soup.find_all('ul')
specifc = img_urls.find()

[<ul>
 <li><a href="https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg" target="_blank">Sample</a> (jpg) 1024px wide</li>
 <li><a href="https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif" target="_blank">Original</a> (tif<span class="tooltip word-tif" title=""></span>) 21 MB</li>
 </ul>,
 <ul class="page-numbers"><li><a class="active" href="//search/map/Mars/Viking/cerberus_enhanced?p=1&amp;pb=1#downloads">1</a></li><li><a class="" href="//search/map/Mars/Viking/cerberus_enhanced?p=2&amp;pb=1#downloads">2</a></li></ul>]